# Multiple Linear Regression <a class="tocSkip">

In [ ]:
# Import statements
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import pandas as pd

%matplotlib inline
%load_ext nb_black

## Motivation

The goal is to find the linear function that best desribes miles-per-gallon for a car given the cylinder, horespower and weight of the car. 

In [ ]:
cars = pd.read_csv(
    "https://raw.githubusercontent.com/mwaskom/seaborn-data/master/mpg.csv"
)
cars = cars[["mpg", "cylinders", "horsepower", "weight"]]
cars = cars.dropna()
cars = (cars - cars.mean()) / cars.std()
cars.head()

## Hypothesis

**Notation**

* **n**:    Number of features
* **m**:    Number of training examples
* **x**:    Input variable / features
* **y**:    Output variable / target



**Hypothesis Function**

$h_\theta = \theta_0 + \theta_1x_1 + \theta_2x_2 + \theta_3x_3 + ... + \theta_nx_n$

By adding a row of ones to X everything can be written in matrix form.

$$y = \begin{bmatrix}y_0\\ y_1\\ .\\.\\ y_m\end{bmatrix}  X = \begin{bmatrix}1 & X_{1,1} & X_{1,2}  &  .&.& X_{1,n}\\ 1 & X_{2,1} & . &.  & . &.\\ .&.  &  .&  .&. &. \\ . &  .& . &.  &. & .\\ 1 & X_{m,1} & . &  .& .& X_{m,n}\end{bmatrix} \theta = \begin{bmatrix}\theta_0\\ \theta_1\\ .\\ .\\ 
\theta_m\end{bmatrix}$$

$$ y: m \times 1 \;\;\;\; X: m \times (n+1) \;\;\;\; \theta: (n+1)\times n $$

The new **Hypothesis Function** is

$h_\theta = X\theta$

## Finding the Solution Analytically

Calculate the RSS (residual sum of squares)

$$ \textrm{RSS} = (y-X\theta)^T(y-X\theta) $$
$$ \textrm{RSS} = (y^T-\theta^TX^T)^T(y-X\theta) $$
$$ \textrm{RSS} = y^Ty - y^TX\theta - \theta^TX^Ty + \theta^TX^TX\theta $$

Minimizing RSS

$$ \frac{\partial (\textrm{RSS})}{\partial \theta} = \frac{\partial (y^Ty - y^TX\theta - \theta^TX^Ty + \theta^TX^TX\theta)}{\partial \theta} = 0 $$

$$\frac{\partial (y^Ty)}{\partial \theta} -\frac{\partial (y^TX\theta)}{\partial \theta} -\frac{\partial (\theta^TX^Ty)}{\partial \theta} +\frac{\partial (\theta^TX^TX\theta)}{\partial \theta} = 0 $$
$$ 0 - y^TX - X^Ty^T + 2\theta^TX^TX = 0 $$
$$ 2\theta^TX^TX = 2y^TX$$
$$ \theta = (X^TX)^{-1}X^Ty $$

The analytical solution does not scale well, as inverting a matrix has roughly complexity of $O(n^3)$, therefore numerical approximation methods are used for problems with a large number of features.

## Calculating the best Parameters using Gradient Descent

repeat until convergence 

$$ \theta_j = \theta_j - \alpha\frac{\partial }{\partial \theta_j}J(\theta)$$

## Solving the Example

In [ ]:
# Prepare the data
X = np.array(cars.drop(["mpg"], axis=1))
y = np.array(cars["mpg"])
# Adding the intercept
ones = np.ones([X.shape[0], 1])
X = np.concatenate((ones, X), axis=1)

### Solving the Example Analytically

In [ ]:
# Solving the example using the analytical solution
hat = np.linalg.inv(X.T.dot(X)).dot(X.T).dot(y)
print(hat)

### Solving the Example using Gradient Descent

In [ ]:
def cost_function(x, y, theta):
    # difference between f(x) and y output
    dif = np.dot(x, theta) - y
    cost = np.sum(dif ** 2) / (2 * np.shape(x)[0])
    return dif, cost

In [ ]:
def multivariate_gradient_descent(x, y, theta, learning_rate, num_iterations):
    cost_history = []
    for i in range(num_iterations):
        dif, cost = cost_function(x, y, theta)
        gradient = np.dot(x.transpose(), dif) / np.shape(x)[0]
        theta = theta - learning_rate * gradient
        cost_history.append(cost)
    return theta, cost_history

In [ ]:
# Set Hyperparameter
learning_rate = 0.0001
num_iterations = 1000000

# Initialize all weights as 0
_, num_features = np.shape(X)
initial_theta = np.zeros(num_features)
theta, cost_history = multivariate_gradient_descent(
    X, y, initial_theta, learning_rate, num_iterations
)
print(theta)

In [ ]:
fig, ax = plt.subplots()
ax.plot(np.arange(num_iterations), cost_history, "r")
ax.set_xlabel("Iterations")
ax.set_ylabel("Cost")
ax.set_title("Error vs. Training Epoch")
plt.show()